## **Programa de Pós-Graduação em Computação - INF/UFRGS**
### Disciplina CMP263 - Aprendizagem de Máquina
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br>

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. *


---

<br>

## **Tópico: Interpretabilidade e Explicabilidade de Modelos Preditivos**

<br>

Leitura recomendada: https://christophm.github.io/interpretable-ml-book

<br>

**Objetivo deste notebook**: Explorar o uso de k-fold cross-validation para treinamento e avaliação de modelos preditivos, e otimização de hiperparâmetros.
<br>

---



##**Interpretabilidade de um modelo de Diagnóstico de Câncer de Mama**

Cada instância se refere ao exame de um(a) paciente. Os atributos são computados a partir de uma imagem digitalizada de material coletado de uma massa mamária através de uma punção aspirativa por agulha fina (PAAF). Por intermédio deste procedimento, é possível obter células de uma suspeita de lesão, que usualmente são analisadas com o auxílio de um microscópio pelo médico patologista. Os dados a serem utilizados definem um conjunto de atributos que descrevem as características dos núcleos celulares presentes na imagem, com o intuito de automatizar o processo de análise e definição do diagnóstico provável.

Dez características foram analisadas para cada núcleo celular:

*   raio (média das distâncias do centro aos pontos do perímetro)
*   textura (desvio padrão dos valores de escala de cinza)
*   perímetro
*   área
*   suavidade (variação local nos comprimentos dos raios)
*   compacidade (perímetro^2 / área - 1,0)
*   concavidade (gravidade das porções côncavas do contorno)
*   pontos côncavos (número de porções côncavas do contorno)
*   simetria
*   dimensão fractal

Para cada característica foram extraídas a média, o erro padrão e o pior (ou maior) valor, resultando em 30 atributos para cada exame. A última coluna, 'diagnosis', contém a classe verdadeira de cada instância, que pode ser M (maligno) ou B (benigno).





---



In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Carregar dados
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

Criação de conjunto de treino e teste, e treinamento dos modelos

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Regressão logística (modelo interpretável)
pipe_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=1000))
])
pipe_lr.fit(X_train, y_train)

# Random Forest (modelo caixa-preta)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
print("Logistic Regression:")
print(classification_report(y_test, pipe_lr.predict(X_test)))

print("Random Forest:")
print(classification_report(y_test, rf.predict(X_test)))


Análise de relevância de atributos pela regressão logística (coeficientes)

In [ ]:
coef = pipe_lr.named_steps['lr'].coef_[0]
coef_df = pd.DataFrame({'Feature': X.columns, 'Coefficient': coef})
coef_df = coef_df.sort_values(by='Coefficient', key=np.abs, ascending=False)
coef_df.head(30)


Análise de relevância de atributos pelo random forests (feature_importances_)

In [ ]:
importances = rf.feature_importances_
importances_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
importances_df = importances_df.sort_values(by='Importance', ascending=False)
importances_df.head(10)

SHAP

In [ ]:
!pip install shap

import shap
## como o processo de calcular valores SHAP é custoso, usamos aqui apenas uma amostra dos dados
X_train_sample = shap.utils.sample(X_train, 50)
# calcula valores de SHAP pro modelo
explainer = shap.Explainer(rf.predict, X_train_sample,feature_names=X_train.columns)
shap_values = explainer(X_test)

In [ ]:
shap.plots.bar(shap_values[0]) ## semelhante ao plots.waterfall.

In [ ]:
#shap.plots.bar(shap_values[0]) ## semelhante ao plots.waterfall.
shap.plots.waterfall(shap_values[0], max_display=10) ##analisa a primeira instância de teste (0)

In [ ]:
# Explicação global  - para um conjunto de instâncias
shap.plots.beeswarm(shap_values, max_display=15)

In [ ]:
!pip install lime
import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X.columns,
    class_names=data.target_names,
    mode='classification'
)

# Explicação de uma instância
i = 0  # índice da instância a explicar
exp = explainer.explain_instance(X_test.iloc[i], rf.predict_proba, num_features=10)
exp.show_in_notebook()